In [66]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
import time 

In [67]:
# instantiate options 
options = webdriver.ChromeOptions() 
 
# run browser in headless mode 
options.headless = True 
 
# load website 
url = "https://www.set.or.th/en/market/statistics/program-trading-value"

In [68]:
def get_lsited_number(text):
    words = text.split()
    numbers = [float(word.replace(',', '')) 
                    for word in words 
                            if (word.replace(',', '').replace('.', '').replace('-', '').isdigit() 
                                or 
                            word.replace(',', '').replace('.', '').replace('-', '').lstrip('+-').isdigit())]
    return numbers


def get_text_only(text):
    words = text.split()
    # text_only = [word for word in words if not any(char.isdigit() for char in word)]
    text_only = [word for word in words if any(char.isalpha() for char in word)]
    return text_only


In [69]:
def update_df(main_df, new_df):
    for col in new_df.columns:
        if col == 'datetime':
            continue
        # print('col:',col, new_df[col].values[0])
        if col in main_df.columns:
            main_df.loc[new_df['datetime'], col] = new_df[col].values[0]
        else:
            # print('new col')
            main_df.loc[new_df['datetime'], col] = new_df[col].values[0]
    main_df = main_df.fillna(0)
    return main_df

In [71]:
# instantiate driver 
driver = webdriver.Chrome(service=ChromeService( 
	ChromeDriverManager().install()), options=options) 

# get the entire website content 
driver.get(url) 
 
# select elements by class name 
elements = driver.find_elements(By.CLASS_NAME, 'table-responsive')
print('len elements = ', len(elements))

for idx, element in enumerate(elements):
    print(idx, get_lsited_number(element.text), get_text_only(element.text))

len elements =  4
0 [] []
1 [18420.32, 18101.94, 318.37, 24128.25, 24446.63, -318.37, 42548.57, 42548.57] ['Type', 'Buy', 'Sell', 'Net', 'Program', 'Trading', 'Non', 'Program', 'Trading', 'Total']
2 [] []
3 [194659662.5, 31.67, 46920037.0, 26.96, 21227720.0, 26.7, 28663545.0, 22.66, 14058629.0, 13.44, 14000473.0, 8.01, 1459231.0, 1.33] ['Symbol', 'Market', 'Program', 'Trading', 'Value', '(Baht)', 'Program', 'Trading', 'Value', 'Comparing', 'with', 'Auto', 'Matching', 'BAM', 'SET', 'BYD', 'SET', 'SAT', 'SET', 'ITC', 'SET', 'MCA', 'mai', 'TGE', 'SET', 'TPS', 'mai']


In [75]:
import pandas as pd

from datetime import datetime

# Define column names for the DataFrames
columns_df1 = [
    'Program Trading Buy',
    'Program Trading Sell',
    'Program Trading Net',
    'Non Program Trading Buy',
    'Non Program Trading Sell',
    'Non Program Trading Net',
    'Total Buy',
    'Total Sell'
]

items_to_delete = ['Symbol', 'Market', 'Program', 'Trading', 'Value', '(Baht)', '%', 'Program', 
                   'Trading', 'Value', 'Comparing', 'with', 'Auto', 'Matching', 'SET', 'mai']


def get_numbers_and_text_from_driver(df1, df2, print_value=False, formatted_date=datetime.now()):
    # select elements by class name 
    elements = driver.find_elements(By.CLASS_NAME, 'table-responsive')
    
    for idx, element in enumerate(elements):
        if idx == 1:
            values = get_lsited_number(element.text)
            if print_value:
                print('\t', len(values), values)
            if len(values)==8:
                df_temp = pd.DataFrame([values], columns=columns_df1)
                df_temp['datetime'] = [formatted_date]
                for col in df1.columns:
                    df1.loc[df_temp['datetime'], col] = df_temp[col].values[0]
        elif idx == 3:
            today_symbols = get_text_only(element.text)
            # print('\t', today_symbols)
            today_symbols = [item for item in today_symbols if item not in items_to_delete]
            values = get_lsited_number(element.text)[1::2]
            if print_value:
                print('\t', len(today_symbols), today_symbols)
                print('\t', len(values), values)
            if len(values)>=1:
                df_temp = pd.DataFrame([values], columns=today_symbols)
                df_temp['datetime']=[formatted_date]
                print(len(df_temp.columns))
                df2 = update_df(df2, df_temp)

    return df1, df2

In [ ]:
from tqdm import tqdm

df1 = pd.DataFrame(columns=columns_df1)
df2 = pd.DataFrame(columns=[])


from datetime import datetime, timedelta
start_date = datetime.now() - timedelta(days=1)
num_days = 120
formatted_dates = []
for i in range(num_days):
    current_date = start_date - timedelta(days=i)
    formatted_date = current_date.strftime("%A, %B %d, %Y")
    formatted_date = formatted_date.replace(" 0", " ")  # Remove leading zero for single digit days
    formatted_dates.append(formatted_date)

print(formatted_dates[0], ' to ',formatted_dates[-1])

df1['datetime'] = formatted_dates
df1.set_index('datetime', inplace=True)
df2['datetime'] = formatted_dates
df2.set_index('datetime', inplace=True)

# instantiate options 
options = webdriver.ChromeOptions() 
# run browser in headless mode 
options.headless = True 
# instantiate driver 
driver = webdriver.Chrome(service=ChromeService( 
	ChromeDriverManager().install()), options=options) 
driver.get(url)


print('START')
skip_origin = False
# Now iterate over the list of formatted dates and perform Selenium actions
for formatted_date in tqdm(formatted_dates):
    if datetime.strptime(formatted_date, "%A, %B %d, %Y").weekday() >= 5:  # Saturday (5) or Sunday (6)
        print(f">>>> SKIP WEEKEND: {formatted_date}")
        continue  # Skip to the next iteration of the loop
    
    print(formatted_date)
    origin = driver.find_element(By.XPATH, "//input[@placeholder='Select date']")
    if not skip_origin:
        origin.click()
        time.sleep(1)
    skip_origin = False
    try:
        day_element_xpath = f"//span[@aria-label='{formatted_date}']"
        day_element = driver.find_element(By.XPATH, day_element_xpath)
        day_element.click()
        time.sleep(1) 
        df1, df2 = get_numbers_and_text_from_driver(df1, df2, True, formatted_date=formatted_date)
    except Exception as e:
        print("An error occurred:", e)
        print(f'>>>>>> ERROR: {formatted_date}.')
        break
    
    # Check if the date part includes " 1,"
    if " 1," in formatted_date:
        # 1st date >>> CHANGE MONTH"
        origin = driver.find_element(By.XPATH,"//input[@placeholder='Select date']")
        origin.click()
        time.sleep(1)
        month_backward = driver.find_element(By.XPATH, "//div[@class='vc-arrow is-left']//*[name()='svg']")
        month_backward.click()
        skip_origin = True
        # BACKWARD COMPLETE
        time.sleep(1)

In [93]:
display(df1)
display(df2)
display(df2.loc['Friday, February 23, 2024'])

,Program Trading Buy,Program Trading Sell,Program Trading Net,Non Program Trading Buy,Non Program Trading Sell,Non Program Trading Net,Total Buy,Total Sell
datetime,,,,,,,,
"Wednesday, March 6, 2024",18420.32,18101.94,318.37,24128.25,24446.63,-318.37,42548.57,42548.57
"Tuesday, March 5, 2024",18005.36,19899.67,-1894.31,23580.46,21686.15,1894.31,41585.82,41585.82
"Monday, March 4, 2024",18801.78,19297.62,-495.84,23540.58,23044.74,495.84,42342.36,42342.36
"Sunday, March 3, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Saturday, March 2, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
"Sunday, November 12, 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Saturday, November 11, 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Friday, November 10, 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,BAM,BYD,SAT,ITC,MCA,TGE,TPS,TKN,JTS,PIN,...,BA,AOT,AQUA,TRP,QLT,FORTH,PR9,WHART,CPNREIT,MOSHI
datetime,,,,,,,,,,,,,,,,,,,,,
"Wednesday, March 6, 2024",31.67,26.96,26.7,22.66,13.44,8.01,1.33,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Tuesday, March 5, 2024",0.00,0.00,0.0,0.00,0.00,7.23,0.00,22.1,19.09,17.69,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Monday, March 4, 2024",0.00,0.00,0.0,0.00,14.47,13.13,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Sunday, March 3, 2024",0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Saturday, March 2, 2024",0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Sunday, November 12, 2023",0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Saturday, November 11, 2023",0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Friday, November 10, 2023",0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


BAM         0.0
BYD         0.0
SAT         0.0
ITC         0.0
MCA        17.3
           ... 
FORTH       0.0
PR9         0.0
WHART       0.0
CPNREIT     0.0
MOSHI       0.0
Name: Friday, February 23, 2024, Length: 223, dtype: float64

In [103]:
df1 = df1.fillna(0)
df2 = df2.fillna(0)
df1.to_csv('df1.csv')
df2.to_csv('df2.csv')

df1_cleaned = df1[~(df1==0.0).all(axis=1)]
df2_cleaned = df2[~(df2==0.0).all(axis=1)]
df1_cleaned.to_csv('df1_cleaned.csv')
df2_cleaned.to_csv('df2_cleaned.csv')
display(df1_cleaned)
display(df2_cleaned)

,Program Trading Buy,Program Trading Sell,Program Trading Net,Non Program Trading Buy,Non Program Trading Sell,Non Program Trading Net,Total Buy,Total Sell
datetime,,,,,,,,
"Wednesday, March 6, 2024",18420.32,18101.94,318.37,24128.25,24446.63,-318.37,42548.57,42548.57
"Tuesday, March 5, 2024",18005.36,19899.67,-1894.31,23580.46,21686.15,1894.31,41585.82,41585.82
"Monday, March 4, 2024",18801.78,19297.62,-495.84,23540.58,23044.74,495.84,42342.36,42342.36
"Friday, March 1, 2024",21325.10,23448.84,-2123.75,30387.55,28263.80,2123.75,51712.65,51712.65
"Thursday, February 29, 2024",36053.82,41805.79,-5751.98,39729.82,33977.84,5751.98,75783.63,75783.63
...,...,...,...,...,...,...,...,...
"Tuesday, November 21, 2023",16269.28,15942.47,326.81,22768.60,23095.41,-326.81,39037.88,39037.88
"Monday, November 20, 2023",15232.25,16010.90,-778.65,24368.46,23589.81,778.65,39600.71,39600.71
"Friday, November 17, 2023",18583.47,19244.19,-660.72,27893.17,27232.45,660.72,46476.64,46476.64


,BAM,BYD,SAT,ITC,MCA,TGE,TPS,TKN,JTS,PIN,...,BA,AOT,AQUA,TRP,QLT,FORTH,PR9,WHART,CPNREIT,MOSHI
datetime,,,,,,,,,,,,,,,,,,,,,
"Wednesday, March 6, 2024",31.67,26.96,26.7,22.66,13.44,8.01,1.33,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
"Tuesday, March 5, 2024",0.00,0.00,0.0,0.00,0.00,7.23,0.00,22.1,19.09,17.69,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
"Monday, March 4, 2024",0.00,0.00,0.0,0.00,14.47,13.13,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
"Friday, March 1, 2024",0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,15.60,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
"Thursday, February 29, 2024",0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Tuesday, November 21, 2023",0.00,26.54,0.0,0.00,0.00,0.00,0.00,0.0,14.84,0.00,...,0.0,0.0,0.0,0.0,0.0,25.24,0.00,0.00,0.00,0.00
"Monday, November 20, 2023",0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
"Friday, November 17, 2023",0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,12.48,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,16.13,15.34,9.12,0.00


# Example of symbol in each day

In [10]:
sym_1 = []
sym_1.append("//div[contains(text(),'TKN')]")
sym_1.append("//div[contains(text(),'JTS')]")

sym_1_value = []
sym_1_value.append("//td[normalize-space()='76,067,330.00']")
sym_1_value.append("//td[normalize-space()='57,999,975.00']")

print(sym_1)
print(sym_1_value)

["//div[contains(text(),'TKN')]", "//div[contains(text(),'JTS')]"]
["//td[normalize-space()='76,067,330.00']", "//td[normalize-space()='57,999,975.00']"]


In [6]:
sym_2 =[]
sym_2.append("//div[contains(text(),'CBG')]")
sym_2.append("//div[contains(text(),'ONEE')]")

sym_2

["//div[contains(text(),'CBG')]", "//div[contains(text(),'ONEE')]"]

In [19]:
# for idx, element in enumerate(elements):
    # print(idx, get_lsited_number(element.text), get_text_only(element.text))

print(get_text_only(elements[-1].text))


['Symbol', 'Market', 'Program', 'Trading', 'Value', '(Baht)', '%', 'Program', 'Trading', 'Value', 'Comparing', 'with', 'Auto', 'Matching', 'TKN', 'SET', 'JTS', 'SET', 'PIN', 'SET', 'XO', 'mai', 'ETL', 'SET', 'SUPER', 'SET', 'PANEL', 'mai', 'TGE', 'SET']


# Extract Symbol Trading Data

In [7]:
# instantiate driver 
driver = webdriver.Chrome(service=ChromeService( 
	ChromeDriverManager().install()), options=options) 

# get the entire website content 
driver.get(url) 
 
# select elements by class name 
elements = driver.find_elements(By.CLASS_NAME, 'table-responsive')
print('len elements = ', len(elements))

for idx, element in enumerate(elements):
    print(idx, get_lsited_number(element.text), get_text_only(element.text))

len elements =  4
0 [] []
1 [18420.32, 18101.94, 318.37, 24128.25, 24446.63, -318.37, 42548.57, 42548.57] ['Type', 'Buy', 'Sell', 'Net', 'Program', 'Trading', 'Non', 'Program', 'Trading', 'Total']
2 [] []
3 [194659662.5, 31.67, 46920037.0, 26.96, 21227720.0, 26.7, 28663545.0, 22.66, 14058629.0, 13.44, 14000473.0, 8.01, 1459231.0, 1.33] ['Symbol', 'Market', 'Program', 'Trading', 'Value', '(Baht)', '%', 'Program', 'Trading', 'Value', 'Comparing', 'with', 'Auto', 'Matching', 'BAM', 'SET', 'BYD', 'SET', 'SAT', 'SET', 'ITC', 'SET', 'MCA', 'mai', 'TGE', 'SET', 'TPS', 'mai']


In [219]:
df2 = pd.DataFrame(columns=columns_df2)
df2

""


In [118]:
df2

,datetime,BAM_x,BYD_x,SAT_x,ITC_x,MCA_x,TGE_x,TPS_x,BAM_y,BYD_y,...,MCA_x,TGE_x,TPS_x,BAM_y,BYD_y,SAT_y,ITC_y,MCA_y,TGE_y,TPS_y
0,"Thursday, March 7, 2024",31.67,26.96,26.7,22.66,13.44,8.01,1.33,31.67,26.96,...,13.44,8.01,1.33,31.67,26.96,26.7,22.66,13.44,8.01,1.33


In [249]:
from datetime import datetime, timedelta
# df2 = pd.DataFrame(columns=columns_df2)
# Get today's date
start_date = datetime.now()
formatted_date = start_date.strftime("%A, %B %d, %Y")
formatted_date = formatted_date.replace(" 0", " ")  # Remove leading zero for single digit days



In [296]:
df1 = pd.DataFrame(columns=columns_df1)
df2 = pd.DataFrame(columns=[])

df1['datetime'] = formatted_dates
df1.set_index('datetime', inplace=True)

df2['datetime'] = formatted_dates
df2.set_index('datetime', inplace=True)


,Program Trading Buy,Program Trading Sell,Program Trading Net,Non Program Trading Buy,Non Program Trading Sell,Non Program Trading Net,Total Buy,Total Sell
datetime,,,,,,,,
"Wednesday, March 6, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Tuesday, March 5, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Monday, March 4, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Sunday, March 3, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Saturday, March 2, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
"Wednesday, November 22, 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Tuesday, November 21, 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Monday, November 20, 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


""
datetime
"Wednesday, March 6, 2024"
"Tuesday, March 5, 2024"
"Monday, March 4, 2024"
"Sunday, March 3, 2024"
"Saturday, March 2, 2024"
...
"Wednesday, November 22, 2023"
"Tuesday, November 21, 2023"
"Monday, November 20, 2023"


In [306]:
df1, df2 = get_numbers_and_text_from_driver(df1, df2)
print('result df1')
display(df1)
print('result df2')
display(df2)

,Program Trading Buy,Program Trading Sell,Program Trading Net,Non Program Trading Buy,Non Program Trading Sell,Non Program Trading Net,Total Buy,Total Sell,datetime
0,18420.32,18101.94,318.37,24128.25,24446.63,-318.37,42548.57,42548.57,"Saturday, November 18, 2023"


result df1


,Program Trading Buy,Program Trading Sell,Program Trading Net,Non Program Trading Buy,Non Program Trading Sell,Non Program Trading Net,Total Buy,Total Sell
datetime,,,,,,,,
"Wednesday, March 6, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Tuesday, March 5, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Monday, March 4, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Sunday, March 3, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Saturday, March 2, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
"Wednesday, November 22, 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Tuesday, November 21, 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Monday, November 20, 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


result df2


,BAM,BYD,SAT,ITC,MCA,TGE,TPS
datetime,,,,,,,
"Wednesday, March 6, 2024",0.00,0.00,0.0,0.00,0.00,0.00,0.00
"Tuesday, March 5, 2024",0.00,0.00,0.0,0.00,0.00,0.00,0.00
"Monday, March 4, 2024",0.00,0.00,0.0,0.00,0.00,0.00,0.00
"Sunday, March 3, 2024",0.00,0.00,0.0,0.00,0.00,0.00,0.00
"Saturday, March 2, 2024",0.00,0.00,0.0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...
"Wednesday, November 22, 2023",0.00,0.00,0.0,0.00,0.00,0.00,0.00
"Tuesday, November 21, 2023",0.00,0.00,0.0,0.00,0.00,0.00,0.00
"Monday, November 20, 2023",0.00,0.00,0.0,0.00,0.00,0.00,0.00


# Create function for scrapping data until last date in csv dataframe

In [74]:
import pandas as pd
from datetime import datetime, timedelta

def from_last_date(file_path):
    # Get today's date
    today_date = datetime.now().strftime("%A, %B %d, %Y").replace(" 0", " ")
    
    # Read the DataFrame and get the last date
    read_df = pd.read_csv(file_path)
    last_date = read_df['datetime'][0]
    
    # Convert last date string to datetime object
    date1 = datetime.strptime(last_date, "%A, %B %d, %Y")
    
    # Convert today's date string to datetime object
    date2 = datetime.strptime(today_date, "%A, %B %d, %Y")
    
    # Calculate the difference in days
    num_days = abs((date2 - date1).days)
    
    # Initialize start date
    start_date = datetime.now()
    formatted_dates = []
    
    # Generate formatted dates for each day
    for i in range(num_days):
        current_date = start_date - timedelta(days=i)
        formatted_date = current_date.strftime("%A, %B %d, %Y")
        formatted_date = formatted_date.replace(" 0", " ")  # Remove leading zero for single digit days
        formatted_dates.append(formatted_date)
    
    return formatted_dates

# Example usage
file_path = './rpa/data_programming.csv'
formatted_dates = from_last_date(file_path)
print(len(formatted_dates))
print(formatted_dates[0], ' to ', formatted_dates[-1])
print(formatted_dates)


0


IndexError: list index out of range

In [84]:
import pandas as pd

df1 = pd.read_csv('./rpa/data_programming.csv')
df2 = pd.read_csv('./rpa/df2_cleaned_new.csv')

# Identify common columns
common_cols = df1.columns.intersection(df2.columns)

# Create new DataFrames with zeros for missing columns
new_df1 = pd.DataFrame(0.0, index=df1.index, columns=common_cols.difference(df1.columns))
new_df2 = pd.DataFrame(0.0, index=df2.index, columns=common_cols.difference(df2.columns))

# Concatenate DataFrames
result = pd.concat([df1, new_df1], axis=1)
result2 = pd.concat([df2, new_df2], axis=1)
result_final = pd.concat([result2, result], axis=0).fillna(0.0, inplace=False)

display(df2)
display(result_final)

,datetime,VGI,STA,AAV,BWG,CHASE,TEGH,TPOLY,TRUBB,CMO
0,"Thursday, March 14, 2024",26.39,25.52,23.29,17.51,14.94,13.54,9.42,9.12,2.29


,datetime,VGI,STA,AAV,BWG,CHASE,TEGH,TPOLY,TRUBB,CMO,...,CPALL,BA,AOT,AQUA,QLT,FORTH,PR9,WHART,CPNREIT,MOSHI
0,"Thursday, March 14, 2024",26.39,25.52,23.29,17.51,14.94,13.54,9.42,9.12,2.29,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
0,"Wednesday, March 13, 2024",25.63,0.00,0.00,0.00,0.00,0.00,5.49,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
1,"Tuesday, March 12, 2024",0.00,0.00,0.00,14.02,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
2,"Monday, March 11, 2024",0.00,0.00,0.00,17.22,0.00,0.00,6.16,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
3,"Friday, March 8, 2024",0.00,0.00,0.00,0.00,24.46,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,"Tuesday, November 21, 2023",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,25.24,0.00,0.00,0.00,0.00
77,"Monday, November 20, 2023",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
78,"Friday, November 17, 2023",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,16.13,15.34,9.12,0.00
79,"Thursday, November 16, 2023",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,15.71


In [85]:
display(pd.read_csv('./rpa/df2_cleaned.csv'))

,datetime,VGI,SKY,ETL,BE8,GIFT,RABBIT,CV,TPOLY,CNT,...,CPALL,BA,AOT,AQUA,QLT,FORTH,PR9,WHART,CPNREIT,MOSHI
0,"Wednesday, March 13, 2024",25.63,24.34,20.56,19.26,17.24,12.59,11.02,5.49,1.03,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
1,"Tuesday, March 12, 2024",0.00,0.00,0.00,0.00,19.42,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
2,"Monday, March 11, 2024",0.00,0.00,0.00,16.64,0.00,0.00,0.00,6.16,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
3,"Friday, March 8, 2024",0.00,0.00,13.88,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
4,"Thursday, March 7, 2024",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,"Tuesday, November 21, 2023",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,25.24,0.00,0.00,0.00,0.00
77,"Monday, November 20, 2023",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
78,"Friday, November 17, 2023",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,16.13,15.34,9.12,0.00
79,"Thursday, November 16, 2023",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,15.71
